<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex10-Fama-French-Five.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Investment Analysis***, Bates, Boyer, and Fletcher  

# Example Chapter 10: Using the Fama-French Five Factor Model
The purpose of this example is to enable you to estimate the Fama-French Five alpha relative to any trading strategy. We first load in the returns from the low beta strategy from the class repo.  You could similarly load in the returns from any strategy that you have save on Google Drive, or your own GitHub repo. We then load in the returns on the FF5 factors. Finally, we estimate the alpha of the trading strategy relative to the five factors.   

### Imports and Setup

In [ ]:
#import packages
# Load in simple_finance.py from the GitHub repository
!curl -O https://raw.githubusercontent.com/boyerb/Investments/master/functions/simple_finance.py
import simple_finance as sf
import pandas as pd
import statsmodels.api as sm

### Load in the Strategy Returns
In the code block below we load in the returns from the low beta strategy.  These are the returns of stocks in the lowest decile when sorted by beta.  Recall that the CAPM alpha of the strategy was negative, and we could reject the hypothesis that the CAPM alpha is zero with 95% confidence. After loading the data, we format the date column for merging with the FF5 data, and set the date to be the index of the DataFrame.  

In [ ]:
# Load in the data by first specifying the URL where the data can be found
url="https://github.com/boyerb/Investments/raw/master/Examples_3.42.xlsx"
columns_to_read = ["date","Low Beta"]
# read the data into a DataFrame
df = pd.read_excel(url, sheet_name="10-LoBeta", header=0, usecols=columns_to_read, engine="openpyxl")
df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y%m').dt.to_period('M') # format date column
df.set_index('date', inplace=True) # set the date column as the index
print(df.head())

### Load in Fama French Five Factors
#### Function Description  
**`get_ff5(start_date`** (optional)**`, end_date`** (optional)**)-> pd.DataFrame`**     
Imports historical monthly returns for the Fama-French-Five factors from Ken French's website and returns the data as a DataFrame  

**Inputs**  
- `start_date` (*string*, optional):  starting month in format `'yyyy-mm'`  
- `end_date` (*string*, optional): ending month in format `'yyyy-mm'`  
If `start_date` and `end_date` are not specified, the function loads in all available data.  

**Returns**  
- (*DataFrame*): Returns for FF5 factors
  - *Index* : Monthly date index (`pandas.PeriodIndex` with `freq='M'`).
 - *MKT-RF* : The excess return on the value-weighted market portfolio
 - *SMB* : Returns on small-minus-big
 - *HML* : Returns on high-minus-low
 - *RMW* : Returns on robust-minus-weak
 - *CAM* : Returns on conservative-minus-aggressive
 - *RF* : The return on short-term t-bills

**Example Usage**:  
`dat=sf.get_ff5()`  
Returns all data for the FF5 Factors

In [ ]:
FF5=sf.get_ff5() # get all data for FF5 factors
print(FF5.head())

### Merge the Two DataFrames on Date
In the block of code below we merge `df` with `FF5` by `date`, which is the index for both DataFrames. The merge is an inner join meaning that any rows in either DataFrame that do not have a match in the other will be deleted.   

In [ ]:
# Perform left join
merged_df = df.merge(FF5, how='inner', left_index=True, right_index=True)
print(merged_df.head())

### Estimate FF5 Alpha
Can we reject that alpha is zero with 95% confidence?

In [ ]:
merged_df['ex_LoBeta'] = merged_df['Low Beta'] - merged_df['RF'] # excess return on low beta strategy
X=merged_df[["Mkt-RF","SMB","HML","RMW","CMA"]] # explanatory variables
Y=merged_df["ex_LoBeta"] # dependent variable
X=sm.add_constant(X) # we specify that we want to add a constant to the regression equation
model=sm.OLS(Y,X).fit() # run regression
params = model.params # pull out parameters
conf_int = model.conf_int(alpha=0.05)  # 95% CIs
r_squared = model.rsquared
# create a table of output
results_df = pd.DataFrame({
    'Parameter': params.index, # 1st column: parameter name
    'Estimate': params.values, # 2nd column: parameer value
    'CI Lower': conf_int[0].values, # 3rd column: lower bound on 95\% CI
    'CI Upper': conf_int[1].values  # 4th column: upper bound on 95\% CI
})

print(results_df)